In [1]:
import os

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import pandas as pd
import numpy as np
import datetime as date


# Configuración de entorno y modificación del Dataset


En este apartado pasaremos a modificar el Dataset según nos parezcan razonables la presencia de ciertas columnas y, 
por otro lado, pasaremos a cambiar de formato los valores nominales por cardinales y, estos a su vez, a valores decimales,
ya que se entiende que de esta forma se favorece al funcionamiento y rendimiento del aprendizaje por regresión lineal.

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
df = pd.read_csv(os.path.join("..", "in", "fifa.csv"))
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,https://cdn.sofifa.org/teams/2/light/241.png,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,https://cdn.sofifa.org/teams/2/light/73.png,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,https://cdn.sofifa.org/teams/2/light/11.png,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,https://cdn.sofifa.org/teams/2/light/10.png,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


Tendremos en cuenta qué columnas tienen valores NaN para después normalizarlos.

In [3]:
df.columns[df.isna().any()].tolist()

['Club',
 'Preferred Foot',
 'International Reputation',
 'Weak Foot',
 'Skill Moves',
 'Work Rate',
 'Body Type',
 'Real Face',
 'Position',
 'Jersey Number',
 'Joined',
 'Loaned From',
 'Contract Valid Until',
 'Height',
 'Weight',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM',
 'CAM',
 'RAM',
 'LM',
 'LCM',
 'CM',
 'RCM',
 'RM',
 'LWB',
 'LDM',
 'CDM',
 'RDM',
 'RWB',
 'LB',
 'LCB',
 'CB',
 'RCB',
 'RB',
 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'GKDiving',
 'GKHandling',
 'GKKicking',
 'GKPositioning',
 'GKReflexes',
 'Release Clause']

Se pasa a eliminar del Dataset las columnas correspondientes al dato 'Unnamed', al nombre, al identificador, a la foto del futbolista, a la bandera, y al logo del club, puesto que se entiende que:

* La columna 'Unnamed', el identificador son números arbitrarios que poco pueden aportar al valor del futbolista per se.


* El nombre es, de nuevo, un modo de identificar al futbolista y no de apreciar su valor.


* La foto del futbolista, si bien podría determinar su valor, este no es cuantificable sin realizar un análisis a la propia imagen, con un criterio y métodos que escapan al análisis actual.


* La foto y escudo del club al que pertenecen los futbolistas son particularidades de dicho club, al que ya se le puede identificar solo por su nombre.

In [4]:
df = df.iloc[:,3:]
df = df.drop(['Photo', 'Flag', 'Club Logo'], axis = 1)
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,€110.5M,€565K,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,33,Portugal,94,94,Juventus,€77M,€405K,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,26,Brazil,92,93,Paris Saint-Germain,€118.5M,€290K,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,27,Spain,91,93,Manchester United,€72M,€260K,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,27,Belgium,91,92,Manchester City,€102M,€355K,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,€60K,€1K,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,22.0,"May 3, 2017",NaN,2019,5'9,134lbs,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,€143K
18203,19,Sweden,47,63,Trelleborgs FF,€60K,€1K,1098,Right,1.0,2.0,2.0,Medium/ Medium,Normal,No,ST,21.0,"Mar 19, 2018",NaN,2020,6'3,170lbs,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40.0,41.0,39.0,38.0,40.0,52.0,41.0,

Se pasa a modificar el formato de los precios presentados en las columnas de Value (valor del futbolista), Wage (salario), y Release Clause (importe cláusula de liberación) para que estén representados de manera uniforme y cardinalmente.

In [5]:
def monetary_value_refactor(x):
    x = str(x).replace('€','')   # Se elimina el carácter €
    val = x
    if 'K' in x:
        if len(x) > 1:
            val = float(x.replace('K', '')) # K se trata como valor * 1.000
        val = val *1000.0
    if 'M' in x:
        if len(x) > 1:
            val = float(x.replace('M', '')) # M se trata como valor * 1.000.000
        val = val * 1000000.0
    return val

df["Value"] = df["Value"].apply(monetary_value_refactor)
df["Wage"] = df["Wage"].apply(monetary_value_refactor)
df["Release Clause"] = df["Release Clause"].apply(monetary_value_refactor)
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159lbs,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183lbs,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150lbs,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168lbs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,60000.0,1000.0,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,22.0,"May 3, 2017",NaN,2019,5'9,134lbs,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,143000.0
18203,19,Sweden,47,63,Trelleborgs FF,60000.0,1000.0,1098,Right,1.0,2.0,2.0,Medium/ Medium,Normal,No,ST,21.0,"Mar 19, 2018",NaN,2020,6'3,170lbs,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2,23.0

Se pasa a cambiar el formato del peso como con los valores de precios anteriormente.

In [6]:
#3
def weight_value_refactor(x):
    return float(str(x).replace('lbs','')) * 1.0  # Se eliminan los caracteres 'lbs' y se pasa a decimal

df["Weight"] = df["Weight"].apply(weight_value_refactor)
df


,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,5'7,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,6'2,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,5'9,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,6'4,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,5'11,154.0,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,60000.0,1000.0,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,22.0,"May 3, 2017",NaN,2019,5'9,134.0,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,143000.0
18203,19,Sweden,47,63,Trelleborgs FF,60000.0,1000.0,1098,Right,1.0,2.0,2.0,Medium/ Medium,Normal,No,ST,21.0,"Mar 19, 2018",NaN,2020,6'3,170.0,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2,23.0,52.0,5

Se pasa a cambiar el formato de la altura como en el caso previo.

In [7]:
def height_feet_to_meters(x):
    if str(x) == "0": return 0.0
    x = str(x).split("'",1)                       # Se separan los valores por '
    if len(x) < 2: return float(x[0])*0.3048
    z = float(x[0])*0.3048+float(x[1])*0.0254  # Se pasan a metros los valores de pies * 0,3048 y pulgadas * 0,0254
    return z

# Sustituimos los valores NaN para facilitar la conversión
df["Height"] = df["Height"].fillna(0.0)
df["Height"] = df["Height"].apply(height_feet_to_meters)
df
    

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,"Jul 1, 2004",NaN,2021,1.7018,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,"Jul 10, 2018",NaN,2022,1.8796,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,"Aug 3, 2017",NaN,2022,1.7526,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,"Jul 1, 2011",NaN,2020,1.9304,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,"Aug 30, 2015",NaN,2023,1.8034,154.0,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,60000.0,1000.0,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,22.0,"May 3, 2017",NaN,2019,1.7526,134.0,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,143000.0
18203,19,Sweden,47,63,Trelleborgs FF,60000.0,1000.0,1098,Right,1.0,2.0,2.0,Medium/ Medium,Normal,No,ST,21.0,"Mar 19, 2018",NaN,2020,1.9050,170.0,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,3

Con respecto a la fecha de contratación, se pretende dar un valor numérico pasando las fechas a días absolutos. Para ello primero necesitamos una fecha de referencia a partir de la que medir los valores, que en este caso será la fecha de contratación más reciente, de manera que este valor pasará a ser el número de días de antigüedad. Cuanto más tardía la fecha, más días de experiencia como futbolista, pero también más desgaste.

In [8]:
ex = pd.DataFrame(columns = ["Aux"])          
ex["Aux"] = df["Joined"].astype('datetime64[ns]')
ex["Aux"].max()  # Fecha de unión más reciente tomada como fecha de actualidad

Timestamp('2018-12-20 00:00:00')

Cambiamos el formato

In [9]:
def date_format_to_days_joined(x):
    if x == 0.0: return 0.0
    y = date.datetime(2018,12,20)  # La fecha de contratación más reciente
    x = x.replace(',','')
    data = date.datetime.strptime(x, '%b %d %Y')
    days = y-data       # Calculamos la antiguedad claculando los días entre que se unió y la fecha de actualidad
    return float(days.days)

# Sustituimos los valores NaN para facilitar la conversión
df["Joined"] = df["Joined"].fillna(0.0)
df["Joined"] = df["Joined"].apply(date_format_to_days_joined)
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,5285.0,NaN,2021,1.7018,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,163.0,NaN,2022,1.8796,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,504.0,NaN,2022,1.7526,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,2729.0,NaN,2020,1.9304,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,1208.0,NaN,2023,1.8034,154.0,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,60000.0,1000.0,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,22.0,596.0,NaN,2019,1.7526,134.0,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,143000.0
18203,19,Sweden,47,63,Trelleborgs FF,60000.0,1000.0,1098,Right,1.0,2.0,2.0,Medium/ Medium,Normal,No,ST,21.0,276.0,NaN,2020,1.9050,170.0,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.0,25.0,40

Para la fecha de 'Contract Valid Until' (caducidad de contrato), se sigue una lógica como la anterior. Se les resta la fecha de referencia y ese tiempo se pasa a días hasta fin de contrato.

In [10]:
df["Contract Valid Until"].unique()

array(['2021', '2022', '2020', '2023', '2019', '2024', 'Jun 30, 2019',
       '2025', '2026', 'Dec 31, 2018', '2018', nan, 'May 31, 2020',
       'Jun 30, 2020', 'May 31, 2019', 'Dec 31, 2019', 'Jan 1, 2019',
       'Jun 1, 2019', 'Jan 4, 2019', 'Jan 31, 2019', 'Jan 7, 2019',
       'Jan 2, 2019', 'Jan 6, 2019', 'Oct 14, 2019', 'Jan 3, 2019',
       'May 4, 2019', 'Jan 12, 2019', 'Jan 25, 2019', 'Jan 18, 2019',
       'Dec 1, 2019', 'Nov 30, 2018', 'Feb 27, 2020', 'Jan 5, 2019',
       'Jan 15, 2019', 'Jan 30, 2019', 'Jan 11, 2019', 'Jan 20, 2019'],
      dtype=object)

Teniendo en cuenta que que hay años sin meses ni días especificados, realizamos el cambio de formato

In [11]:
def date_format_to_days_left(x):
    if x == 0.0: return 0.0
    val = 0
    if len(x) <= 4:
        y = date.datetime(2018,12,20)
        days = date.datetime(2019,1,1) - y  # Asignamos los días hasta 2019 por defecto
        years = int(x) - 2019
        val = days.days + years*365         # Sumamos los días hasta 2019 más los días hasta el año especificado
    return float(val)
    z = date.datetime(2018,12,20)
    x = x.replace(',','')
    data = date.datetime.strptime(x, '%b %d %Y')
    days2 = y-data                                # Restamos las fechas y devolvemos los días
    return float(days2.days)

# Sustituimos los valores NaN para facilitar la conversión
df["Contract Valid Until"] = df["Contract Valid Until"].fillna(0.0)
df["Contract Valid Until"] = df["Contract Valid Until"].apply(date_format_to_days_left)
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,Left,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,5285.0,NaN,742.0,1.7018,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,Right,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,163.0,NaN,1107.0,1.8796,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,Right,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,504.0,NaN,1107.0,1.7526,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,Right,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,2729.0,NaN,377.0,1.9304,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,Right,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,1208.0,NaN,1472.0,1.8034,154.0,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,60000.0,1000.0,1307,Right,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,22.0,596.0,NaN,12.0,1.7526,134.0,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,143000.0
18203,19,Sweden,47,63,Trelleborgs FF,60000.0,1000.0,1098,Right,1.0,2.0,2.0,Medium/ Medium,Normal,No,ST,21.0,276.0,NaN,377.0,1.9050,170.0,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2,23.0,52.0,52.0,43.0,36.0,39.0,32.0,20.

En el caso de la columna' Preferred Foot' (pie preferido), dado que los valores son 'Left' (izquierdo) o 'Right' (derecho), se ha pasado a clasificarlos como dos clases diferentes para pasar esta información nominal a cardinal

In [12]:
pref_foot = df.pop("Preferred Foot")

df = pd.concat([df.reset_index(drop=True), 
                pd.get_dummies(pref_foot, prefix='Preferred_Foot_', dtype=float).reset_index(drop=True)], axis=1, sort=False)
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Preferred_Foot__Left,Preferred_Foot__Right
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,5.0,4.0,4.0,Medium/ Medium,Messi,Yes,RF,10.0,5285.0,NaN,742.0,1.7018,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0,1.0,0.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,5.0,4.0,5.0,High/ Low,C. Ronaldo,Yes,ST,7.0,163.0,NaN,1107.0,1.8796,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0,0.0,1.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,5.0,5.0,5.0,High/ Medium,Neymar,Yes,LW,10.0,504.0,NaN,1107.0,1.7526,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0,0.0,1.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,4.0,3.0,1.0,Medium/ Medium,Lean,Yes,GK,1.0,2729.0,NaN,377.0,1.9304,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0,0.0,1.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,4.0,5.0,4.0,High/ High,Normal,Yes,RCM,7.0,1208.0,NaN,1472.0,1.8034,154.0,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,60000.0,1000.0,1307,1.0,2.0,2.0,Medium/ Medium,Lean,No,CM,22.0,596.0,NaN,12.0,1.7526,134.0,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,143000.0,0.0,1.0
18203,19,Sweden,47,63,Trelleborgs FF,60000.0,1000.0,1098,1.0,2.0,2.0,Medium/ Medium,Normal,No,ST,21.0,276.0,NaN,377.0,1.9050,170.0,45+2,45+2,45+2,39+2,42+2,42+2,42+2,39+2,40+2,40+2,40+2,38+2,35+2,35+2,35+2,38+2,30+2,31+2,31+2,31+2,30+2,29+2,32+2,32+2,32+2,29+2

Con respecto a la columna 'Body Type' (tipo de cuerpo), dado que se tratan valores nominales, se hará una clasificación para pasar esta información a cardinal.

In [13]:
df["Body Type"].unique()

array(['Messi', 'C. Ronaldo', 'Neymar', 'Lean', 'Normal', 'Courtois',
       'Stocky', 'PLAYER_BODY_TYPE_25', 'Shaqiri', 'Akinfenwa', nan],
      dtype=object)

Viendo los tipos posibles de cuerpo, se pasará a hacer 4 clases, los que tienen el tipo único (los que tienen nombres propios), los normales ('Normal'), los delgados ('Lean'), y los robustos ('Stocky').

In [14]:
# Se hacen copias de 'Body Type' para modificar

unique_body_type = pd.DataFrame(df["Body Type"].values,columns = ["Val"]).copy()
lean_body_type = pd.DataFrame(df["Body Type"].values,columns = ["Val"]).copy()
normal_body_type = pd.DataFrame(df["Body Type"].values,columns = ["Val"]).copy()
stocky_body_type = pd.DataFrame(df["Body Type"].values,columns = ["Val"]).copy()

# Modificamos cada copia según los valores correspondientes por clase

unique_body_type["Val"] = unique_body_type["Val"].replace({'Messi': 1.0, 'C. Ronaldo': 1.0, 'Neymar': 1.0, 
                                                           'Courtois': 1.0, 'Stocky': 0.0, 'PLAYER_BODY_TYPE_25': 1.0, 
                                                           'Shaqiri': 1.0, 'Akinfenwa': 1.0, 'Lean': 0.0,
                                                           'Normal': 0.0, 'nan': 0.0})
lean_body_type["Val"] = lean_body_type["Val"].replace({'Messi': 0.0, 'C. Ronaldo': 0.0, 'Neymar': 0.0, 'Courtois': 0.0, 
                                                       'Stocky': 0.0, 'PLAYER_BODY_TYPE_25': 0.0, 'Shaqiri': 0.0, 
                                                       'Akinfenwa': 0.0, 'Lean': 1.0, 'Normal': 0.0, 'nan': 0.0})
normal_body_type["Val"] = normal_body_type["Val"].replace({'Messi': 0.0, 'C. Ronaldo': 0.0, 'Neymar': 0.0, 'Courtois': 0.0, 
                                                           'Stocky': 0.0, 'PLAYER_BODY_TYPE_25': 0.0, 'Shaqiri': 0.0, 
                                                            'Akinfenwa': 0.0, 'Lean': 0.0, 'Normal': 1.0, 'nan': 0.0})
stocky_body_type["Val"] = stocky_body_type["Val"].replace({'Messi': 0.0, 'C. Ronaldo': 0.0, 'Neymar': 0.0, 'Courtois': 0.0, 
                                                           'Stocky': 1.0, 'PLAYER_BODY_TYPE_25': 0.0, 'Shaqiri': 0.0, 
                                                            'Akinfenwa': 0.0, 'Lean': 0.0, 'Normal': 0.0, 'nan': 0.0})

# Se insertan las nuevas columnas ya modificadas

df.insert(df.columns.get_loc("Body Type"), "Unique Body Type",unique_body_type["Val"].values.tolist())
df.insert(df.columns.get_loc("Body Type"), "Lean Body Type",lean_body_type["Val"].values.tolist())
df.insert(df.columns.get_loc("Body Type"), "Normal Body Type",normal_body_type["Val"].values.tolist())
df.insert(df.columns.get_loc("Body Type"), "Stocky Body Type",stocky_body_type["Val"].values.tolist())

# Borramos la columna 'Body Type'
df = df.drop(columns = ["Body Type"])
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Work Rate,Unique Body Type,Lean Body Type,Normal Body Type,Stocky Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Preferred_Foot__Left,Preferred_Foot__Right
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,5.0,4.0,4.0,Medium/ Medium,1.0,0.0,0.0,0.0,Yes,RF,10.0,5285.0,NaN,742.0,1.7018,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0,1.0,0.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,5.0,4.0,5.0,High/ Low,1.0,0.0,0.0,0.0,Yes,ST,7.0,163.0,NaN,1107.0,1.8796,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0,0.0,1.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,5.0,5.0,5.0,High/ Medium,1.0,0.0,0.0,0.0,Yes,LW,10.0,504.0,NaN,1107.0,1.7526,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0,0.0,1.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,4.0,3.0,1.0,Medium/ Medium,0.0,1.0,0.0,0.0,Yes,GK,1.0,2729.0,NaN,377.0,1.9304,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0,0.0,1.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,4.0,5.0,4.0,High/ High,0.0,0.0,1.0,0.0,Yes,RCM,7.0,1208.0,NaN,1472.0,1.8034,154.0,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,60000.0,1000.0,1307,1.0,2.0,2.0,Medium/ Medium,0.0,1.0,0.0,0.0,No,CM,22.0,596.0,NaN,12.0,1.7526,134.0,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.0,38.0,40.0,49.0,25.0,42.0,30.0,34.0,45.0,43.0,54.0,57.0,60.0,49.0,76.0,43.0,55.0,40.0,47.0,38.0,46.0,46.0,39.0,52.0,43.0,45.0,40.0,48.0,47.0,10.0,13.0,7.0,8.0,9.0,143000.0,0.0,1.0
18203,19,Sweden,47,63,Trelleborgs FF,60000.0,1000.0,1098,1.0,2.0,2.0,Medium/ Medium,0.0,0.0,1.0,0.0,No,ST,21.0,276.0,NaN,377.0,1.9050,170

La columna 'Work Rate' (ritmo de trabajo), se expresa de manera similar que la de 'Body Type', por lo que seguirá el mismo proceso.

In [15]:
df["Work Rate"].unique()

array(['Medium/ Medium', 'High/ Low', 'High/ Medium', 'High/ High',
       'Medium/ High', 'Medium/ Low', 'Low/ High', 'Low/ Medium',
       'Low/ Low', nan], dtype=object)

Se entiende que cada valor respresenta una clase diferente.

In [16]:
#repetimos el proceso de clasificación para nueve posibles valores llevado a nueve clasificaciones con valores binarios
work_rate = df.pop("Work Rate")

df = pd.concat([df.reset_index(drop=True), 
                pd.get_dummies(work_rate, prefix='Work_Rate', dtype=float).reset_index(drop=True)], axis=1, sort=False)
df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Unique Body Type,Lean Body Type,Normal Body Type,Stocky Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Preferred_Foot__Left,Preferred_Foot__Right,Work_Rate_High/ High,Work_Rate_High/ Low,Work_Rate_High/ Medium,Work_Rate_Low/ High,Work_Rate_Low/ Low,Work_Rate_Low/ Medium,Work_Rate_Medium/ High,Work_Rate_Medium/ Low,Work_Rate_Medium/ Medium
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,5.0,4.0,4.0,1.0,0.0,0.0,0.0,Yes,RF,10.0,5285.0,NaN,742.0,1.7018,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,5.0,4.0,5.0,1.0,0.0,0.0,0.0,Yes,ST,7.0,163.0,NaN,1107.0,1.8796,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,5.0,5.0,5.0,1.0,0.0,0.0,0.0,Yes,LW,10.0,504.0,NaN,1107.0,1.7526,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,4.0,3.0,1.0,0.0,1.0,0.0,0.0,Yes,GK,1.0,2729.0,NaN,377.0,1.9304,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,4.0,5.0,4.0,0.0,0.0,1.0,0.0,Yes,RCM,7.0,1208.0,NaN,1472.0,1.8034,154.0,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,60000.0,1000.0,1307,1.0,2.0,2.0,0.0,1.0,0.0,0.0,No,CM,22.0,596.0,NaN,12.0,1.7526,134.0,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34.

A priori vemos que la columna 'Real Face' (cara real) puede requerir de clasificación.

In [17]:
df["Real Face"].unique()

array(['Yes', 'No', nan], dtype=object)

Tras ver los posibles valores, directamente pasaremos un 'Yes' (sí) como un 1 y un 'No'  como 0.

In [18]:
def yes_no_numbered(x):
    if str(x) == '0.0': return 0.0
    if x == 'Yes': return 1.0
    if x == 'No': return 0.0

# Sustituimos los valores NaN para facilitar la conversión
df["Real Face"] = df["Real Face"].fillna(0.0)
df["Real Face"] = df["Real Face"].apply(yes_no_numbered)

df

,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Unique Body Type,Lean Body Type,Normal Body Type,Stocky Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Preferred_Foot__Left,Preferred_Foot__Right,Work_Rate_High/ High,Work_Rate_High/ Low,Work_Rate_High/ Medium,Work_Rate_Low/ High,Work_Rate_Low/ Low,Work_Rate_Low/ Medium,Work_Rate_Medium/ High,Work_Rate_Medium/ Low,Work_Rate_Medium/ Medium
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,5.0,4.0,4.0,1.0,0.0,0.0,0.0,1.0,RF,10.0,5285.0,NaN,742.0,1.7018,159.0,88+2,88+2,88+2,92+2,93+2,93+2,93+2,92+2,93+2,93+2,93+2,91+2,84+2,84+2,84+2,91+2,64+2,61+2,61+2,61+2,64+2,59+2,47+2,47+2,47+2,59+2,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,5.0,4.0,5.0,1.0,0.0,0.0,0.0,1.0,ST,7.0,163.0,NaN,1107.0,1.8796,183.0,91+3,91+3,91+3,89+3,90+3,90+3,90+3,89+3,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,5.0,5.0,5.0,1.0,0.0,0.0,0.0,1.0,LW,10.0,504.0,NaN,1107.0,1.7526,150.0,84+3,84+3,84+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,89+3,88+3,81+3,81+3,81+3,88+3,65+3,60+3,60+3,60+3,65+3,60+3,47+3,47+3,47+3,60+3,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,4.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,GK,1.0,2729.0,NaN,377.0,1.9304,168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,4.0,5.0,4.0,0.0,0.0,1.0,0.0,1.0,RCM,7.0,1208.0,NaN,1472.0,1.8034,154.0,82+3,82+3,82+3,87+3,87+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,87+3,87+3,88+3,77+3,77+3,77+3,77+3,77+3,73+3,66+3,66+3,66+3,73+3,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,60000.0,1000.0,1307,1.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,CM,22.0,596.0,NaN,12.0,1.7526,134.0,42+2,42+2,42+2,44+2,44+2,44+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,44+2,44+2,45+2,45+2,45+2,44+2,45+2,45+2,45+2,45+2,45+2,34

En el caso de las estadísticas con potenciales, para pasar la suma de dos números, entendiendo el segundo sumando como un potencial y que este, sumado directamente al primer sumando podría dar lugar  a que dos valores inicialmente diferente con diferente pontencial tuviesen el mismo valor: 45+3 = 46+2; pero pensando que no tienen el mismo valor para valorar el precio del futbolista, se pasará a sumar la mitad del potencial -> 46.5 /= 47. 

Esto podría producir el mismo error que simplemente sumando los dos valores para otros casos, pero se cree que este error sería considerablemente menor.

In [19]:
def add(x):
    z = x
    if len(str(x)) > 2 and str(x) != "0.0":
        y = str(x).split("+",1)
        z = int(y[0])+int(y[1])/2
    return float(z)

slice_cols = df.loc[::,"LS":"RB"].columns.tolist()  # Se listan los nombres de las estadísticas con potenciales

for x in slice_cols:                        # Para cada estadística se modifica según lo antes descrito
    df[x].fillna(0.0, inplace = True)
    df[x] = df[x].apply(add)

df


,Age,Nationality,Overall,Potential,Club,Value,Wage,Special,International Reputation,Weak Foot,Skill Moves,Unique Body Type,Lean Body Type,Normal Body Type,Stocky Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause,Preferred_Foot__Left,Preferred_Foot__Right,Work_Rate_High/ High,Work_Rate_High/ Low,Work_Rate_High/ Medium,Work_Rate_Low/ High,Work_Rate_Low/ Low,Work_Rate_Low/ Medium,Work_Rate_Medium/ High,Work_Rate_Medium/ Low,Work_Rate_Medium/ Medium
0,31,Argentina,94,94,FC Barcelona,110500000.0,565000.0,2202,5.0,4.0,4.0,1.0,0.0,0.0,0.0,1.0,RF,10.0,5285.0,NaN,742.0,1.7018,159.0,89.0,89.0,89.0,93.0,94.0,94.0,94.0,93.0,94.0,94.0,94.0,92.0,85.0,85.0,85.0,92.0,65.0,62.0,62.0,62.0,65.0,60.0,48.0,48.0,48.0,60.0,84.0,95.0,70.0,90.0,86.0,97.0,93.0,94.0,87.0,96.0,91.0,86.0,91.0,95.0,95.0,85.0,68.0,72.0,59.0,94.0,48.0,22.0,94.0,94.0,75.0,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,226500000.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,33,Portugal,94,94,Juventus,77000000.0,405000.0,2228,5.0,4.0,5.0,1.0,0.0,0.0,0.0,1.0,ST,7.0,163.0,NaN,1107.0,1.8796,183.0,92.5,92.5,92.5,90.5,91.5,91.5,91.5,90.5,89.5,89.5,89.5,89.5,82.5,82.5,82.5,89.5,66.5,62.5,62.5,62.5,66.5,62.5,54.5,54.5,54.5,62.5,84.0,94.0,89.0,81.0,87.0,88.0,81.0,76.0,77.0,94.0,89.0,91.0,87.0,96.0,70.0,95.0,95.0,88.0,79.0,93.0,63.0,29.0,95.0,82.0,85.0,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,127100000.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,26,Brazil,92,93,Paris Saint-Germain,118500000.0,290000.0,2143,5.0,5.0,5.0,1.0,0.0,0.0,0.0,1.0,LW,10.0,504.0,NaN,1107.0,1.7526,150.0,85.5,85.5,85.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,90.5,89.5,82.5,82.5,82.5,89.5,66.5,61.5,61.5,61.5,66.5,61.5,48.5,48.5,48.5,61.5,79.0,87.0,62.0,84.0,84.0,96.0,88.0,87.0,78.0,95.0,94.0,90.0,96.0,94.0,84.0,80.0,61.0,81.0,49.0,82.0,56.0,36.0,89.0,87.0,81.0,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,228100000.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,27,Spain,91,93,Manchester United,72000000.0,260000.0,1471,4.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,GK,1.0,2729.0,NaN,377.0,1.9304,168.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,13.0,21.0,50.0,13.0,18.0,21.0,19.0,51.0,42.0,57.0,58.0,60.0,90.0,43.0,31.0,67.0,43.0,64.0,12.0,38.0,30.0,12.0,68.0,40.0,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,138600000.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,27,Belgium,91,92,Manchester City,102000000.0,355000.0,2281,4.0,5.0,4.0,0.0,0.0,1.0,0.0,1.0,RCM,7.0,1208.0,NaN,1472.0,1.8034,154.0,83.5,83.5,83.5,88.5,88.5,88.5,88.5,88.5,89.5,89.5,89.5,89.5,88.5,88.5,88.5,89.5,78.5,78.5,78.5,78.5,78.5,74.5,67.5,67.5,67.5,74.5,93.0,82.0,55.0,92.0,82.0,86.0,85.0,83.0,91.0,91.0,78.0,76.0,79.0,91.0,77.0,91.0,63.0,90.0,75.0,91.0,76.0,61.0,87.0,94.0,79.0,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,196400000.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18202,19,England,47,65,Crewe Alexandra,60000.0,1000.0,1307,1.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,CM,22.0,596.0,NaN,12.0,1.7526,134.0,43.0,43.0,43.0,45.0,45.0,45.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,45.0,45.0,46.0,46.0,46.0,45.0,46.0,46.0,46.0,46.0,46.0,34

En este punto se considera que todos los datos restantes por analizar forman parte de las características de un futbolista y que aportan valor a la hora de predecir valor. 

En concreto el número de jersey en 'Jersey Number' puede servir como justificativo para los porteros al tener estos valores nulos en las estadísticas potenciadas.

Restaría clasificar los valores en las columnas 'Nationality' (nacionalidad), 'Club', 'Position' (posición), y 'Loaned From' (club de procedencia); y dar formato numérico decimal a todos los valores.

In [20]:

clb = df.pop("Club")
nation = df.pop("Nationality")
position = df.pop("Position")
loaned = df.pop("Loaned From")

df = pd.concat([df.reset_index(drop=True), 
                pd.get_dummies(clb, prefix='club_', dtype=float).reset_index(drop=True)], axis=1, sort=False)
df = pd.concat([df.reset_index(drop=True), 
                pd.get_dummies(nation, prefix='nation_', dtype=float).reset_index(drop=True)], axis=1, sort=False)
df = pd.concat([df.reset_index(drop=True), 
                pd.get_dummies(position, prefix='position_', dtype=float).reset_index(drop=True)], axis=1, sort=False)
df = pd.concat([df.reset_index(drop=True), 
                pd.get_dummies(loaned, prefix='loaned_from_', dtype=float).reset_index(drop=True)], axis=1, sort=False)
df

Age  Overall  Potential        Value      Wage  Special  \
0       31       94         94  110500000.0  565000.0     2202   
1       33       94         94   77000000.0  405000.0     2228   
2       26       92         93  118500000.0  290000.0     2143   
3       27       91         93   72000000.0  260000.0     1471   
4       27       91         92  102000000.0  355000.0     2281   
...    ...      ...        ...          ...       ...      ...   
18202   19       47         65      60000.0    1000.0     1307   
18203   19       47         63      60000.0    1000.0     1098   
18204   16       47         67      60000.0    1000.0     1189   
18205   17       47         66      60000.0    1000.0     1228   
18206   16       46         66      60000.0    1000.0     1321   

       International Reputation  Weak Foot  Skill Moves  Unique Body Type  \
0                           5.0        4.0          4.0               1.0   
1                           5.0        4.0          5.0               1.0   
2                           5.0        5.0          5.0               1.0   
3                           4.0        3.0          1.0               0.0   
4                           4.0        5.0          4.0               0.0   
...                         ...        ...          ...               ...   
18202                       1.0        2.0          2.0               0.0   
18203                       1.0        2.0          2.0               0.0   
18204                       1.0        3.0          2.0               0.0   
18205                       1.0        3.0          2.0               0.0   
18206                       1.0        3.0          2.0               0.0   

       Lean Body Type  Normal Body Type  Stocky Body Type  Real Face  \
0                 0.0               0.0               0.0        1.0   
1                 0.0               0.0               0.0        1.0   
2                 0.0               0.0               0.0        1.0   
3                 1.0               0.0               0.0        1.0   
4                 0.0               1.0               0.0        1.0   
...               ...               ...               ...        ...   
18202             1.0               0.0               0.0        0.0   
18203             0.0               1.0               0.0        0.0   
18204             0.0               1.0               0.0        0.0   
18205             1.0               0.0               0.0        0.0   
18206             1.0               0.0               0.0        0.0   

       Jersey Number  Joined  Contract Valid Until  Height  Weight    LS  \
0               10.0  5285.0                 742.0  1.7018   159.0  89.0   
1                7.0   163.0                1107.0  1.8796   183.0  92.5   
2               10.0   504.0                1107.0  1.7526   150.0  85.5   
3                1.0  2729.0                 377.0  1.9304   168.0   0.0   
4                7.0  1208.0                1472.0  1.8034   154.0  83.5   
...              ...     ...                   ...     ...     ...   ...   
18202           22.0   596.0                  12.0  1.7526   134.0  43.0   
18203           21.0   276.0                 377.0  1.9050   170.0  46.0   
18204           33.0   537.0                 742.0  1.7272   148.0  46.0   
18205           34.0   240.0                  12.0  1.7780   154.0  48.0   
18206           33.0    51.0                  12.0  1.7780   176.0  44.0   

         ST    RS    LW    LF    CF    RF    RW   LAM   CAM   RAM    LM   LCM  \
0      89.0  89.0  93.0  94.0  94.0  94.0  93.0  94.0  94.0  94.0  92.0  85.0   
1      92.5  92.5  90.5  91.5  91.5  91.5  90.5  89.5  89.5  89.5  89.5  82.5   
2      85.5  85.5  90.5  90.5  90.5  90.5  90.5  90.5  90.5  90.5  89.5  82.5   
3       0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4      83.5  83.5  88.5  88.5  88.5  88.5  88.5  89.5  89.5  89.5  89.5  88.5   
...     ...   ...   ...  

Vemos qué columnas requieren de cambiar de formato a decimal.

In [21]:
df.select_dtypes(exclude=[float])

,Age,Overall,Potential,Value,Wage,Special,Release Clause
0,31,94,94,110500000.0,565000.0,2202,226500000.0
1,33,94,94,77000000.0,405000.0,2228,127100000.0
2,26,92,93,118500000.0,290000.0,2143,228100000.0
3,27,91,93,72000000.0,260000.0,1471,138600000.0
4,27,91,92,102000000.0,355000.0,2281,196400000.0
...,...,...,...,...,...,...,...
18202,19,47,65,60000.0,1000.0,1307,143000.0
18203,19,47,63,60000.0,1000.0,1098,113000.0
18204,16,47,67,60000.0,1000.0,1189,165000.0
18205,17,47,66,60000.0,1000.0,1228,143000.0


Cambiamos formato.

In [22]:
columns = df.select_dtypes(exclude=[float]).columns.tolist()

for column in columns:
    df[column] = df[column].astype(float)

df.select_dtypes(exclude=[float])

""
0
1
2
3
4
...
18202
18203
18204
18205


Por último, nos aseguramos de que no haya ningún valor ausente que pueda impedir un correcto funcionamiento del sistema de predicción.

In [23]:
#Confirmamos que no haya valores NaN
#Limpiamos todos los posibles valores NaN

na_columns = df.columns[df.isna().any()].tolist()

for column in na_columns:
    df[column].fillna(0.0, inplace=True)

df.columns[df.isna().any()].tolist() 

[]

# Predicción

En este apartado se pasará a utilizar una regresión lineal para que analice nuestro modelo, de manera que pueda predecir los valores de la columna 'Value', que sería el precio del futbolista, con el menor error posible.

Comenzamos dividiendo nuestras muestras entre un 33% para el testeo y el 67% restante para entrenamiento.

In [24]:
val = df.pop("Value")
X_train, X_test, y_train, y_test = train_test_split(df, val, test_size=0.33, random_state=42)

Aparte de darle un formato numérico y decimal a todos los valores para favorecer el funcionamiento y desempeño del análisis predictivo, también se ha pasado a estandarizar todos los valores empleando un escalado por característica, que distribuirá todos los valores de cada columna entre -1 y 1.

In [25]:
scaler = MinMaxScaler(feature_range = (-1.00,1.00)) 
scaled_X_train = scaler.fit_transform(X_train)      # Escalamos los datos de entrenamiento

reg = linear_model.LinearRegression().fit(scaled_X_train, y_train)  # Entrenamos nuestro modelo

scaled_X_test = scaler.fit_transform(X_test)       # Escalamos los datos de testeo

preds = reg.predict(scaled_X_test)                 # Llevamos a cabo la predicción

Ejemplos de precios predichos:

In [26]:
preds

array([1346192.,  284144.,  429072., ..., 1804112., 2599280.,   86608.])

Ejemplos de precios empleados para comparar en la fase de testeo:

In [27]:
y_test

5848     1200000.0
14634     270000.0
11807     400000.0
5826      775000.0
1989     9000000.0
           ...    
1626     5000000.0
12568     400000.0
6557     1100000.0
5333     2500000.0
7685      650000.0
Name: Value, Length: 6009, dtype: float64

Calculamos el error mediante el coeficiente de determinación.

In [28]:
r2_score(y_true=y_test, y_pred= preds)

-212789343755243.1

Al ver que se nos devuelve un valor negativo y excesivamente alto, entendemos que podemos atribuir esto a dos cuestiones:

* El rango de los valores es demasiado amplio y se deberían escalar los precios también.

* El modelo contempla 1274, que es una dimensionalidad muy amplia que podría comprometer la calidad de las predicciones.

A priori, entenderemos que se debe al rango de los valores, por lo que pasaremos a recalcular el coeficiente tras escalar los valores.

##### Escalado

In [29]:
scaled_y_test = scaler.fit_transform(y_test.values.reshape(-1,1))
scaled_y_pred = scaler.fit_transform(preds.reshape(-1,1))

r2_score(y_true=scaled_y_test, y_pred= scaled_y_pred)

-0.9312974154551539

Al ver que se nos da un valor negativo, aunque cercano a 0, entendemos que nuestro modelo es ampliamente mejorable. 

Se considera, por tanto, que la mejor manera de hacerlo sería reduciendo el número de características tenidas en cuenta, que ahora mismo serían 1274.

Para ello, en vez de hacer selecciones arbitrarias, se plantea el uso del análisis principal de componente (PCA en inglés) como manera de reducir la dimensionalidad de las muestras.

In [30]:
pca_scale = PCA(n_components = 1207) # Se asgina el número de carácteristicas a tener en cuenta

pca_train_1 = pca_scale.fit_transform(scaled_X_train)   # Se redimensiona la muestra de entrenamiento ya escalada
pca_test_1 = pca_scale.transform(scaled_X_test)         # Se redimensiona la muestra de testeo ya escalada
 
reg_1 = linear_model.LinearRegression().fit(pca_train_1, y_train)  # Se entrena el nuevo modelo

preds_1 = reg_1.predict(pca_test_1)    # Se lleva a cabo la predicción

r2_score(y_true= y_test, y_pred= preds_1)   #  Calculamos un nuevo coeficiente de determinación

0.9728143077034803

Al ver que el valor es muy próximo a 1, que sería el mejor valor posible para esta medida, entendemos que se trata de un coeficiente que indica un desempeño destacable a la hora de preveer el precio de un futbolista, teniendo en cuenta que hay 67 características de las 1274 que teníamos, que empeoraban nuestro modelo y, por tanto, sobraban. 

Sin embargo, creemos que sería interesante calcular el mismo valor tras realizar el PCA y escalando los precios, tal y como se hizo anteriormente:

In [31]:
scaled_y_pred_pca = scaler.fit_transform(preds_1.reshape(-1,1))

r2_score(y_true= scaled_y_test, y_pred= scaled_y_pred_pca)

0.6682425219907637

Al ver que tenemos un valor positivo y a medio camino de ser un valor igual de aceptable que el anterior, pero nada despreciable, entendemos que el escalado de los precios puede afectar el cálculo de los errores y que, para el coeficiente de determinación, nos resulta más positivo tener en cuenta el valor obtenido sin escalar los valores, pero tras realizar el PCA

Ahora, veamos qué nos indican el error cuadrático medio y el error absoluto medio:

* MSE sin escalar:

In [32]:
mean_squared_error(y_true= y_test,y_pred= preds_1)

859934693231.0504

* MSE tras escalar:

In [33]:
scaled_y_pred = scaler.fit_transform(preds_1.reshape(-1,1))
mean_squared_error(y_true= scaled_y_test,y_pred= scaled_y_pred)

0.003437804927106126

Como podemos observar, la comparación entre los valores antes y después de escalar son incomparables, por lo que deducimos que, al menos para esta métrica, es necesario escalar, a diferencia del coeficiente de determinación, que si bien los valores son diferentes, no hay tanta distancia entre los mismos.

* MAE antes de escalar:

In [34]:
mean_absolute_error(preds_1,y_test)

498579.6530312054

* MAE tras escalar:

In [35]:
mean_absolute_error(y_true= scaled_y_test,y_pred= scaled_y_pred)

0.056953117404979

Al igual que con el MSE, para el error absoluto medio nos resulta de más provecho utilizar valores escalados.

Ahora bien, viendo los efectos del escalado en los errores, se plantea realizar el mismo procedimiento pero con una normalización mediante estandarización para apreciar las diferencias.

##### Estandarización

In [36]:
standard = StandardScaler()

standardized_X_train = standard.fit_transform(X_train)   # Estandarizamos los datos de entrenamiento
standardized_X_test = standard.fit_transform(X_test)     # Estandarizamos los datos de testeo

reg_stan = linear_model.LinearRegression().fit(standardized_X_train, y_train)
preds_stan = reg_stan.predict(standardized_X_test)

r2_score(preds_stan, y_test)

9.43689570931383e-15

Aquí podemos apreciar un error infinitesimal, pero negativo. Veamos qué ocurre al estandarizar los precios:

In [37]:
stand_y_test = standard.fit_transform(y_test.values.reshape(-1,1))
stand_y_pred = standard.fit_transform(preds_stan.reshape(-1,1))

r2_score(y_true=stand_y_test, y_pred= stand_y_pred)

-0.9962396531010049

Al igual que al escalar, los valores se diferencian, aunque de manera opuesta (de menos a más), pero siguen siendo negativos. Pasemos a aplicar el PCA:

In [38]:
pca_standard = PCA(n_components = 1207)

pca_train_2 = pca_standard.fit_transform(standardized_X_train) # Se redimensiona la muestra de entrenamiento estandarizado
pca_test_2 = pca_standard.transform(standardized_X_test)       # Se redimensiona la muestra de testeo estandarizado

reg_2 = linear_model.LinearRegression().fit(pca_train_2, y_train)
preds_2 = reg_2.predict(pca_test_2)

r2_score(preds_2, y_test)

0.971067116751922

Vemos que el resultado es apenas diferente del obtenido a estandarizar y realizar el PCA. Veamos qué ocurre si estandarizamos los precios tras realizar el PCA:

In [39]:
pca_stand_y_pred = standard.fit_transform(preds_2.reshape(-1,1))

r2_score(y_true=stand_y_test, y_pred= pca_stand_y_pred)

0.9723115435640286

Resulta que los valores apenas se diferencian tras estandarizar, algo que no ocurría si escalábamos, donde, tras escalar los precios, el cálculo del coeficiente de determinación era menos favorable. Veamos qué ocurre si estandarizamos los precios del modelo:

In [40]:
pca_scaled_y_pred = standard.fit_transform(preds_1.reshape(-1,1))

r2_score(y_true=stand_y_test, y_pred= pca_stand_y_pred)

0.9723115435640286

Como podemos apreciar, el valor es exactamente el mismo que el devuelto por el modelo entrenado con valores estandarizados, pero usando valores entrenados tras escalar y tras estandarizar los precios. 

De esto entendemos que quizás resulte más interesante utilizar el coeficiente de determinación con valores estandarizados y no escalados.

También correspondería observar los valores de MSE y MAE tras estandarizar:

* MSE tras estandarizar:

In [41]:
mean_squared_error(y_true= stand_y_test,y_pred= pca_stand_y_pred)

0.027688456435971384

* MAE tras estandarizar:

In [42]:
mean_absolute_error(y_true= stand_y_test,y_pred= pca_stand_y_pred)

0.07812033578482461

Se entiende de los datos que para estas métricas, la diferencia entre normalizar y escalar existe pero no es del todo destacable.

# Conclusión

Tras modificar nuestro Dataset, hemos pasado a realizar una regresión lineal que nos ha mostrado que nuestro primer modelo no era el idóneo para realizar una predicción aceptable de los precios de los futbolistas.

Una vez realizados un preceso de escalado y de selección de características mediante un PCA para nuestro modelo, fuimos capaces de obtener un resultado más que positivo obteniendo:

* $R^2$ = 0.9728143077034803

* Tras $escalar$ los precios, $MSE$ = 0.003437804927106126

* Tras $escalar$ los precios, $MAE$ = 0.056953117404979

Tras ello se quiso ver la diferencia que podría haber entre el escalar o el estandarizar para normalizar los valores del modelo, obteniendo:

* $R^2$ = 0.971067116751922

* Tras $estandarizar$ los precios, $MSE$ = 0.027688456435971384

* Tras $estandarizar$ los precios, $MAE$ = 0.07812033578482461

Se llegó a la conclusión de que:

* Si las cracterísticas a tener en cuenta son las mismas, entrenar el modelo con valores escalados o estandarizados no proporciona una diferencia en el desempeño muy diferente, según indican los valores del $R^2$, $MSE$ y del $MAE$ de ambos modelos.

* A la hora de calcular el  $R^2$ es más interesante que los valores estén estandarizados, que no escalados.

Mostramos ejemplos de las predicciones obtenidas:

In [43]:
preds_1 # entrenamiento mediante valores escalados

array([1346178.72052742,  283978.45956741,  428632.40494598, ...,
       1803498.9280373 , 2599078.96285115,   86514.57283433])

In [44]:
preds_2 # entrenamiento mediante valores estandarizados

array([1564174.89360414,  217528.70822241,  620144.30483813, ...,
       1582821.57239526, 2245707.29985488,  -81097.05017651])

Se adjunta link al repositorio público donde se encuentra este mismo notebook:
https://github.com/APujol97/Practica_IA_FIFA_2018